### Content-based Filtering Recommendation
- google cloud Datalab 실습

In [1]:
import numpy as np
import tensorflow as tf

tf.enable_eager_execution()
print(tf.__version__)

1.13.1


- User List, Movie List, Movie's Features 만들기
- Movie Feature: Action, Sci-Fi, Comedy, Cartoon, Drama

In [2]:
users = ['Ryan', 'Danielle', 'Vijay', 'Chris']
movies = ['Star Wars', 'The Dark Knight', 'Shrek', 'The Incredibles', 'Bleu', 'Memento']
features = ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']

num_users = len(users)
num_movies = len(movies)
num_feats = len(features)
num_recommendation = 2

#### User, Rating, Feature 초기화
- Uses-Movie Rating Matrix (1 ~ 10)
    - 0 은 not seen/rated
- Movie Feature 의 k-hot encoded Matrix
    - Movie Feature Matrix
    - Five Categories

In [6]:
# Users x Movies => 4users x 6movies
users_movies = tf.constant([
    [4,  6,  8,  0, 0, 0],
    [0,  0, 10,  0, 8, 3],
    [0,  6,  0,  0, 3, 7],
    [10, 9,  0,  5, 0, 2]], dtype=tf.float32)

# Movies x Features => 6 movies x 5 features
movies_feats = tf.constant([
    [1, 1, 0, 0, 1],
    [1, 1, 0, 0, 0],
    [0, 0, 1, 1, 0],
    [1, 0, 1, 1, 0],
    [0, 0, 0, 0, 1],
    [1, 0, 0, 0, 1]], dtype=tf.float32)

### User Feature Matrix

In [7]:
users_feats = tf.matmul(users_movies, movies_feats)
users_feats

<tf.Tensor: id=6, shape=(4, 5), dtype=float32, numpy=
array([[10., 10.,  8.,  8.,  4.],
       [ 3.,  0., 10., 10., 11.],
       [13.,  6.,  0.,  0., 10.],
       [26., 19.,  5.,  5., 12.]], dtype=float32)>

In [8]:
users_feats = users_feats/tf.reduce_sum(users_feats, axis=1, keepdims=True)
users_feats

<tf.Tensor: id=10, shape=(4, 5), dtype=float32, numpy=
array([[0.25      , 0.25      , 0.2       , 0.2       , 0.1       ],
       [0.0882353 , 0.        , 0.29411766, 0.29411766, 0.32352942],
       [0.44827586, 0.20689656, 0.        , 0.        , 0.3448276 ],
       [0.3880597 , 0.2835821 , 0.07462686, 0.07462686, 0.17910448]],
      dtype=float32)>

### User 마다 비슷한 Ranking Feature

In [9]:
top_users_features = tf.nn.top_k(users_feats, num_feats)
top_users_features

TopKV2(values=<tf.Tensor: id=13, shape=(4, 5), dtype=float32, numpy=
array([[0.25      , 0.25      , 0.2       , 0.2       , 0.1       ],
       [0.32352942, 0.29411766, 0.29411766, 0.0882353 , 0.        ],
       [0.44827586, 0.3448276 , 0.20689656, 0.        , 0.        ],
       [0.3880597 , 0.2835821 , 0.17910448, 0.07462686, 0.07462686]],
      dtype=float32)>, indices=<tf.Tensor: id=14, shape=(4, 5), dtype=int32, numpy=
array([[0, 1, 2, 3, 4],
       [4, 2, 3, 0, 1],
       [0, 4, 1, 2, 3],
       [0, 1, 4, 2, 3]], dtype=int32)>)